## RESEARCHES FOR MEDICAL DEVICES ON ANVISA WEBSITE

DEVELOPED BY:
##### CARLOS TRAJANO  >>  DATA SCIENTIST   -   PROJECT | PRODUCT | PROCESS ENGINEER   -   BUSINESS MANAGEMENT MBA

MEDICAL DEVICES SPECIALIST

In [1]:
#FRAMEWORKS REQUIRED

from bs4 import BeautifulSoup
import requests
from requests import get
import warnings
warnings.filterwarnings("ignore")
import time
from time import sleep
# webdriver function imported from selenium library
from selenium import webdriver
from selenium.common import exceptions
import pandas as pd
import sqlite3
import glob #Find all the pathnames matching a specified pattern according to the rules
import os #opening files as CMD command prompt
from datetime import date
from datetime import datetime
from typing import List, Dict
import re

In [2]:
# SELECT THE DIRECTORY PATH AND FILE
path = 'D:\PYTHON PROJECTS\AULAS DH\PESQUISA ANVISA PRODUTO'

FILE = 'TA_PRODUTO_SAUDE_SITE.csv'

In [3]:
# CSV FILE

MD_SEARCH = pd.read_csv(
    path + '\\' + FILE, # FILE SELECTED
    engine = 'c',
    encoding = 'ANSI', # ENCODE PATTERN FROM FILE 
    sep= ';',
    )

In [4]:
# OPEN CSV FILE IN SQL 

db_loading = sqlite3.connect(f'{FILE}.db') # CONNECT WITH SQL AND OPEN THE DB FILE

MD_SEARCH.to_sql(
    name = 'MD', #TABLE NAME
    index = False, # INDEX NOT SAVED
    if_exists = 'replace', # WHETHER THE SAME FILE NAME EXISTS, IT MUST BE REPLACED BY THE NEW FILE
    con = db_loading # DATABASE CREATED (CONNECTERD BY)
)

In [8]:
# MD TABLE 

MD = pd.read_sql_query(
    '''
    SELECT numero_registro_cadastro as REGISTRO,
    NOME_TECNICO,
    NOME_COMERCIAL,
    classe_risco AS CLASSE,
    cnpj_detentor_registro_cadastro AS CNPJ,
    detentor_registro_cadastro AS DETENTOR,
    NOME_FABRICANTE,
    nome_pais_fabric AS PAIS_FABRIC,
    numero_processo AS PROCESSO,
    dt_pub_registro_cadastro AS DATA_PUBLIC,
    validade_registro_cadastro AS VALID_REGISTRO,
    dt_atualizacao_dado AS DATA_AT
    
    FROM MD
    ''',
    sqlite3.connect(f'{path}\\{FILE}.db')
)

pd.set_option('display.max_rows', 10)

print(f'''WERE FOUND {MD.shape[0]} REGISTERS IN THE LAST UPADATE, ON {pd.to_datetime(MD['DATA_AT'][0][:10], format = '%d/%m/%Y')}''')
MD

WERE FOUND 94242 REGISTERS IN THE LAST UPADATE, ON 2021-07-10 00:00:00


,REGISTRO,NOME_TECNICO,NOME_COMERCIAL,CLASSE,CNPJ,DETENTOR,NOME_FABRICANTE,PAIS_FABRIC,PROCESSO,DATA_PUBLIC,VALID_REGISTRO,DATA_AT
0,10246810001,TRIGLICERÍDEO,TRIGLICERÍDEOS MÉTODO TOTALMENTE ENZIMÁTICO,II,72807043000194,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,BRASIL,250000000329516,29/02/1996,VIGENTE,10/07/2021 19:00:01
1,10246810011,COLESTEROL TOTAL,COLESTEROL MÉTODO ENZIMÁTICO,II,72807043000194,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,BRASIL,250000000339589,29/02/1996,VIGENTE,10/07/2021 19:00:01
2,10246810007,ÁCIDO ÚRICO,Ácido Úrico Método Enzimático,II,72807043000194,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,BRASIL,250000000349541,29/02/1996,VIGENTE,10/07/2021 19:00:01
3,10246810012,URÉIA,UREIA METODO ENZIMATICO,II,72807043000194,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,BRASIL,250000000359512,29/02/1996,VIGENTE,10/07/2021 19:00:01
4,10246810004,GLICOSE,GLICOSE METODO ENZIMATICO,II,72807043000194,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,LABORLAB PRODUTOS PARA LABORATÓRIOS LTDA EPP,BRASIL,250000000369577,29/02/1996,VIGENTE,10/07/2021 19:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...
94237,81327670011,PROCALCITONINA PCT,Cartucho de teste mLabs PCT,II,22940751000120,LUMIRADX HEALTHCARE LTDA,"MICROPOINT BIOSCIENCE, INC",ESTADOS UNIDOS DA AMÉRICA,25351999982201624,05/09/2016,VIGENTE,10/07/2021 19:00:01
94238,81639889003,Lubrificante Intimo,Sete Sensações,II,5304403000148,NS PRODUTOS E SISTEMAS DE HIGIENE LTDA,NS PRODUTOS E SISTEMAS DE HIGIENE LTDA,BRASIL,25351999998202096,01/10/2020,VIGENTE,10/07/2021 19:00:01
94239,10041120135,Agente de Obturacao Provisoria,CAVITINE - FORRADOR DE CAVIDADES E VERNIZ PARA...,II,68567650000157,SS WHITE ARTIGOS DENTARIOS LTDA,None,None,2599101183677,28/05/1999,VIGENTE,10/07/2021 19:00:01
94240,10042570009,IMUNOHEMATOLOGIA - REAGENTES COMPLEMENTARES,ALBUMINA BOVINA A 22%,I,44058162000156,PROTHEMO PRODUTOS HEMOTERAPICOS LTDA,PROTHEMO PRODUTOS HEMOTERAPICOS LTDA,BRASIL,2599101614878,25/08/1994,VIGENTE,10/07/2021 19:00:01


In [7]:
#CREATE VARIABLES TO TREAT DATA

#CREATE A LIST OF COLUMNS
COLS = list(MD.columns)

#LIST OF UNIQUE STRINGS FOR EACH COLUMN
L_REGISTRO = [MD['REGISTRO'].unique()]
L_NOME_COMERCIAL = list(MD['NOME_COMERCIAL'].unique())
L_CNPJ = list(MD['CNPJ'].unique())
L_DETENTOR = list(MD['DETENTOR'].unique())
L_NOME_FABRICANTE = list(MD['NOME_FABRICANTE'].unique())
L_PAIS_FABRIC = list(MD['PAIS_FABRIC'].unique())
L_DATA_PUBLIC = list(MD['NOME_TECNICO'].unique())
L_VALID_REGISTRO = list(MD['NOME_TECNICO'].unique())

In [13]:
MD.to_dict(orient = 'Dict', columns=['DETENTOR', 'CNPJ'])

#- 'dict' (default) : dict like {column -> {index -> value}}

#df.to_dict(into=OrderedDict)
#OrderedDict([('col1', OrderedDict([('row1', 1), ('row2', 2)])),
#             ('col2', OrderedDict([('row1', 0.5), ('row2', 0.75)]))])

TypeError: to_dict() got an unexpected keyword argument 'columns'

In [16]:
Companies: List[Dict[str, str]]

def search_company(
    Companies: List[Dict[str, str]], company_name: str) -> Dict[str, str]:
    for company in Companies:
        company_name, cnpj = company.values()
        
        if company_name == f'{company_name} {cnpj}':
            return company
        
        regex= re.compile(fr'{company} \s+{cnpj}', flags=re.I)
        if regex.search(company_name):
            return company
    
    return {}

if __company__ = "__main__":
    
        


#for cnpj in L_CNPJ:
#    detentor = (MD.loc[MD['CNPJ']== cnpj, 'DETENTOR'].unique()).str
#    L_DETENTOR[detentor] = cnpj

#L_DETENTOR
#MD['DETENTOR'].unique().shape[0]

#MD.loc[MD['DETENTOR'].str.contains('MEDTRONIC'), 'CNPJ'].unique()

#MD[MD['DETENTOR'].str.contains('MEDTRONIC')]

#df[df['a'].isin(['x'])] buscar palavra ou fragmento
#df[df['a'].str.contains('x')] buscar palavra ou fragmento 
#df[df['a'].str.len()> 9] palavras com quantidade de caract maior que 9
#df[df['a'].str.contains('x|y')] -- buscar varios - - use barra \ para inserir caracteres especiais !, +, -, % ...

In [101]:
#Procurando arquivos que serão utilizados
#CAMINHO DOS ARQUIVOS
path = 'D:\PYTHON PROJECTS\AULAS DH\PESQUISA ANVISA PRODUTO'
file= 'TA_PRODUTO_SAUDE_SITE.csv'
encoding = []

os.chdir(path) #ABRE O CAMINHO DOS ARQUIVOS CONFORME VARIAVEL 'PATH'
os.listdir() #LISTA OS NOMES DOS ARQUIVOS DO CAMINHO ACESSADO
with open(path + '\\' + file) as encoding:
    encoding[]

SyntaxError: invalid syntax (<ipython-input-101-425920576b9f>, line 10)

In [46]:
# URL FROM ANVISA'S WEBSITE -- 

url = 'https://dados.anvisa.gov.br//dados//TA_PRODUTO_SAUDE_SITE.csv'

# Formata a url inserindo os inputs
#search_url = url.format(
#    FILE_REQUIRED = 'TA_PRODUTO_SAUDE_SITE.csv'
#)

#capturamos a resposta que a URL nos trás
response = get(url)
response

SSLError: HTTPSConnectionPool(host='dados.anvisa.gov.br', port=443): Max retries exceeded with url: //dados//TA_PRODUTO_SAUDE_SITE.csv (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))

In [47]:
# SELENIUM -- OPEN THE BROWSER

# VARIABLE WHICH CALL GOOGLE CHROME'S BROWSER THROUGH WEBDRIVER --SELENIUM
options = webdriver.ChromeOptions()

#open the page in main plane -- options to open in second plane (closed -- hidden)
options.add_argument("--start-maximized")

# options.headless = True #COMMAND HIDDEN

#VARIABLE WHICH OPEN CHROME'S EXECUTABLE
path = './chromedriver.exe'

# VARIABLE WHICH OPEN THE BROWSER AND ITS DEFINED OPTIONS
driver = webdriver.Chrome(path, chrome_options = options)

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [49]:
# OPEN THE WEBSITE AND WAIT FOR 10 SEC
driver.get(url)
time.sleep(10) # WAIT FOR 10 SEC

NameError: name 'driver' is not defined

In [ ]:
# Determina url para requisição
url = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/zona-oeste/pinheiros/apartamento_residencial/?pagina={n_pagina}'
# Faz requisição, passando número da pagina como 1
doc = requests.get(url.format(n_pagina=1))

In [ ]:
# Dá uma olhada no conteúdo de doc
doc.content[:500]

In [ ]:
# Utiliza o BeautifulSoup para interpretar o conteúdo de doc como HTML
parsed_doc = BeautifulSoup(doc.content, 'html.parser')

In [ ]:
# Aparentemente, nenhuma grande mudança. Mas...
str(parsed_doc)[:500]

In [ ]:
# Encontra todas as divs cuja classe é js-card-selector e traz como lista
cards = parsed_doc.find_all('div', class_='js-card-selector')
# A quantidade de cartões na página é igual ao tamanho da lista!
len(cards)